In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008, 'o3': 'atmpro'}
band = [7]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.040203                    1 -0.031193
0.000750        2 -0.044582                    2 -0.034637
0.001052        3 -0.053023                    3 -0.041366
0.001476        4 -0.061120                    4 -0.047836
0.002070        5 -0.068884                    5 -0.054097
0.002904        6 -0.076327                    6 -0.060117
0.004074        7 -0.083427                    7 -0.065926
0.005714        8 -0.090179                    8 -0.071456
0.008015        9 -0.096589                    9 -0.076779
0.011243       10 -0.102680                   10 -0.081804
0.015771       11 -0.108597                   11 -0.086602
0.022122       12 -0.114357                   12 -0.091188
0.031031       13 -0.120124                   13 -0.095540
0.043528       14 -0.125842                   14 -0.099597
0.061057       15 -0.127730                   15 -0.100783
0.085645       16 -0.118274                   16 -0.094183
0.120136       17 -0.069359                   17 -0.056649
0.168516       18  0.034707                   18  0.028636
0.236378       19  0.204513                   19  0.174376
0.331549       20  0.452706                   20  0.396239
0.465100       21  0.793114                   21  0.711483
0.652400       22  1.246198                   22  1.137258
0.915100       23  1.773180                   23  1.642744
1.283650       24  2.035283                   24  1.947552
1.800600       25  1.822231                   25  1.810140
2.525700       26  1.459399                   26  1.456028
3.542800       27  1.079162                   27  1.071612
4.969550       28  0.758238                   28  0.767736
6.970850       29  0.534084                   29  0.566448
9.778100       30  0.363348                   30  0.409262
13.715850      31  0.190538                   31  0.231923
19.239350      32  0.021716                   32  0.046254
26.987250      33 -0.096630                   33 -0.088812
37.855300      34 -0.179443                   34 -0.187381
53.100050      35 -0.249011                   35 -0.274127
73.887500      36 -0.271865                   36 -0.311177
97.662500      37 -0.241494                   37 -0.280953
121.437500     38 -0.209562                   38 -0.242180
145.212500     39 -0.195197                   39 -0.219830
168.987500     40 -0.174965                   40 -0.190832
192.762500     41 -0.140473                   41 -0.149692
216.537500     42 -0.111644                   42 -0.117597
240.312500     43 -0.091885                   43 -0.095851
264.087500     44 -0.073657                   44 -0.076175
287.862500     45 -0.059204                   45 -0.060829
311.637500     46 -0.049442                   46 -0.050550
335.412500     47 -0.041021                   47 -0.041695
359.187500     48 -0.033125                   48 -0.033537
382.962500     49 -0.026187                   49 -0.026454
406.737500     50 -0.020065                   50 -0.020233
430.512500     51 -0.014311                   51 -0.014409
454.287500     52 -0.008921                   52 -0.008985
478.062500     53 -0.003873                   53 -0.003891
501.837500     54  0.000852                   54  0.000887
525.612500     55  0.005410                   55  0.005475
549.387500     56  0.009917                   56  0.010021
573.162500     57  0.014456                   57  0.014617
596.937500     58  0.019338                   58  0.019556
620.712500     59  0.024421                   59  0.024731
644.487500     60  0.029710                   60  0.030077
668.262500     61  0.035235                   61  0.035628
692.037500     62  0.040976                   62  0.041412
715.812500     63  0.046993                   63  0.047555
739.587500     64  0.053328                   64  0.054112
763.362500     65  0.059947                   65  0.061087
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -22.302118  0.000000 -22.302118 -21.900354 -5.323193e-08   
0.000624    2     -22.302122  0.000001 -22.302121 -21.900358  1.000679e-06   
0.000876    3     -22.302124  0.000002 -22.302122 -21.900359  1.478644e-06   
0.001229    4     -22.302128  0.000003 -22.302124 -21.900362  2.303544e-06   
0.001723    5     -22.302133  0.000005 -22.302128 -21.900366  3.689159e-06   
0.002417    6     -22.302142  0.000008 -22.302134 -21.900373  5.964951e-06   
0.003391    7     -22.302156  0.000014 -22.302142 -21.900383  9.643988e-06   
0.004757    8     -22.302178  0.000022 -22.302156 -21.900400  1.551153e-05   
0.006672    9     -22.302211  0.000035 -22.302176 -21.900425  2.477471e-05   
0.009359    10    -22.302263  0.000055 -22.302207 -21.900464  3.926443e-05   
0.013128    11    -22.302340  0.000087 -22.302253 -21.900523  6.177660e-05   
0.018415    12    -22.302457  0.000136 -22.302321 -21.900612  9.653502e-05   
0.025830    13    -22.302632  0.000211 -22.302421 -21.900746  1.498934e-04   
0.036232    14    -22.302894  0.000324 -22.302569 -21.900945  2.314412e-04   
0.050823    15    -22.303283  0.000496 -22.302787 -21.901241  3.556329e-04   
0.071291    16    -22.303870  0.000773 -22.303097 -21.901687  5.572060e-04   
0.100000    17    -22.304823  0.001324 -22.303499 -21.902414  9.633717e-04   
0.140271    18    -22.306363  0.002533 -22.303830 -21.903591  1.870241e-03   
0.196760    19    -22.308645  0.005047 -22.303598 -21.905329  3.799990e-03   
0.275997    20    -22.311718  0.010040 -22.301678 -21.907631  7.738357e-03   
0.387100    21    -22.315288  0.019568 -22.295720 -21.910174  1.549762e-02   
0.543100    22    -22.318185  0.037121 -22.281064 -21.911838  3.031033e-02   
0.761700    23    -22.317086  0.068292 -22.248794 -21.909670  5.759450e-02   
1.068500    24    -22.305616  0.121265 -22.184352 -21.898225  1.058573e-01   
1.498800    25    -22.279942  0.199333 -22.080609 -21.873986  1.808989e-01   
2.102400    26    -22.243601  0.293283 -21.950318 -21.839483  2.758359e-01   
2.949000    27    -22.199233  0.395272 -21.803961 -21.796818  3.792049e-01   
4.136600    28    -22.152129  0.499984 -21.652145 -21.750959  4.841153e-01   
5.802500    29    -22.110460  0.607944 -21.502516 -21.709904  5.945799e-01   
8.139200    30    -22.081718  0.727035 -21.354682 -21.680854  7.223375e-01   
11.417000   31    -22.076971  0.863370 -21.213602 -21.674582  8.749907e-01   
16.014700   32    -22.118961  1.009132 -21.109829 -21.714041  1.040774e+00   
22.464000   33    -22.241261  1.148022 -21.093239 -21.833699  1.195772e+00   
31.510500   34    -22.478831  1.282041 -21.196790 -22.072441  1.339332e+00   
44.200100   35    -22.885423  1.418899 -21.466524 -22.493689  1.478885e+00   
62.000000   36    -23.531289  1.539719 -21.991570 -23.192543  1.599675e+00   
85.775000   37    -24.381261  1.624032 -22.757229 -24.154345  1.685012e+00   
109.550000  38    -25.113305  1.675951 -23.437353 -24.999070  1.738402e+00   
133.325000  39    -25.745065  1.717518 -24.027547 -25.723738  1.780943e+00   
157.100000  40    -26.329720  1.752437 -24.577283 -26.378914  1.816943e+00   
180.875000  41    -26.855549  1.785507 -25.070041 -26.950074  1.850603e+00   
204.650000  42    -27.291327  1.825669 -25.465658 -27.410575  1.889480e+00   
228.425000  43    -27.657935  1.877852 -25.780083 -27.790814  1.938494e+00   
252.200000  44    -27.979186  1.940327 -26.038859 -28.119125  1.996831e+00   
275.975000  45    -28.254696  2.008395 -26.246301 -28.397222  2.060371e+00   
299.750000  46    -28.493729  2.080690 -26.413038 -28.636081  2.127900e+00   
323.525000  47    -28.711498  2.159217 -26.552282 -28.851914  2.201352e+00   
347.300000  48    -28.911440  2.243629 -26.667811 -29.048381  2.280380e+00   
371.075000  49    -29.093921  2.332821 -26.7611

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')